
# TP 1 : Theorie des Graphes 1 
#### L3 INFO NEC 2024 –2025
#### Université de Pau et des Pays de l’Adour
###### date: "2024-12-18"

### installer avec pip (sur la machine)

c'est pareil avec Windows, Linux, Macos

#### si vous avez la version python

```sh
pip install graphviz
pip install networkx
```

#### si vous avez la version python3

```sh
pip3 install graphviz
pip3 install networkx
```

#### installer pip :

```sh
python3 -m ensurepip --upgrade
```
enlever 3 dans python3 pour les version ultérieures<br>
si problèmes voir : https://pip.pypa.io/en/stable/installation/



In [1]:
# modules importants pour le(s) tp(s) :
import networkx as nx # type: ignore # partie rendu graphique
import graphviz as gz  # type: ignore
from graphviz import Digraph # type: ignore

# partie système
from time import time, sleep
import os, sys
from random import randint

# partie mathématique + structure
import numpy as np # type: ignore
from math import inf, sqrt, cos, sin, tan


En premier, il faut définir les variables d'entrée: le graphe. <br>
Ce graphe est donné par un vecteur des sommets, une matrice des arêtes, et un vecteur des poids. <br>

Le graphe avec lequel on va tester notre code est simple: <br>
On va avoir 6 sommets et ça c'est la liste de sommets adjacentes avec les poids : 

### on importe nos classes pour créer une structure de graphe

In [2]:
class Vertex:
    """gère les sommets et ses arrêtes (sous forme d'autres sommets)"""

    def __init__(self, name, weight=0):
        """name:le nom du sommet (vertex) associé au noeud
        weight: le poids du sommet pour aller du noeud jusq'au sommet (par défaut=0)
        (nom du noeud non représenté ici)"""
        self.name = name
        self.weight = weight # du sommet du noeud jusq'a ce sommet (ici)
    
    def __str__(self):
        '''si on fait print(Vertex) le resultat du print est dans ce return'''
        return self.name

In [12]:
class Node:
    """
    représente les noeuds entre des point dans un graphe
    
    on crée la classe qui s'occupe de tous les noeuds entre 
    les sommets (vertex/vertecies) et leurs autres sommets reliés
    les arrêtes (edges) ne sont pas représentés car c'est la matrice d'adjacence qui s'occupe de ça
    qui est situé dans la classe Graph
    """

    def __init__(self, node_vertex:Vertex, vertecies:list[Vertex]):
        self.name = node_vertex.name
        self.weight = node_vertex.weight # si jamais on peut revenir vers le sommet lui même (boucle)
        self.vertecies = vertecies

    def __str__(self, print_weight=True):
        '''si on affiche Node (print(Node)) renverra ce qui suit'''
        res = f"{self.name} ["
        v_size = len(self.vertecies)
        for i, e in enumerate(self.vertecies):
            res += f"{e.name}"
            if(print_weight):
                res += f" w={e.weight}"
            if(i!=v_size-1):
                if v_size>1:
                    res += "," 
                res+=" "
        res+="]"
        return res
    
    def _sort(self, vertecies_:list[Vertex])->list[Vertex]:
        """fonction privée a ne pas utiliser (en dehors de la classe)
        algorithme reccursif pour les problèmes de pronfondeur et de performances"""
        if len(vertecies_)<2 :
            return vertecies_
        else:
            # elem du milieu
            pivot = vertecies_[len(vertecies_)//2].weight
            l, m, r = [],[],[] # mineurs, égal, majeurs 
            for v in vertecies_:
                if(v.weight < pivot):
                    l.append(v)
                elif(v.weight == pivot):
                    m.append(v) # si reccursion sur len(m) ce n'est jamais < 2, (boucle infini) dans certains cas
                else: r.append(v)
            return self._sort(l)+m+self._sort(r)
        
    def sort(self):
        '''trie la liste des sommets (vetecies) par poids croissant (ASC)'''
        # on utilise une fonction réccursive donc code separement
        self.vertecies = self._sort(self.vertecies)




In [13]:

class Graph:

    def __init__(self, data:list[Node], title:str="default"):
        self.title = title # titre du graphique si utilisé
        self.matrix = [] # matrice des liens entre les noeuds (Edges)
        self.adj = data # matrice d'adjacence
        self.weight = [[nod.name for nod in node.vertecies] for node in data] # poids des trajets
        self.nodes = [node.name for node in data] # liste des noeuds (seulement les noms)
        self.degree = 0 # pas encore calculé le degré du graphe

    def __str__(self): # OK
        """si jamais on print un graph (print(Graph)) c'est executé ici
        affiche le plus simplement avec des caractères le graph"""
        res = ""
        for i, node in enumerate(self.adj):
            res += f"{i}| {node.name} ["
            v_size = len(node.vertecies)
            for j in range(v_size):
                res += f"{node.vertecies[j].name}"
                if(j!=v_size-1):
                    if v_size>1:
                        res += "," 
                    res+=" "
            res += "]\n"
        return res
    
    def sort_weight_ASC(self): # OK
        """trie le graphe par poids croissants(asc)
        on trie chaque arrêtes du graphe
        """
        for node in self.adj:
            node.vertecies.sort()
    
    def render(self):
        """effectue le rendu du graphe visuellement"""
        graph = Digraph()
        graph.nodes(self.nodes)
        graph.edges(
            [(self.adj[i].name, vertex.name) for i in range(len(self.adj))
            for vertex in self.adj[i].vertecies]
        )
        graph.render(self.title, format='png', view=True)

In [14]:
# test
nodes = Node(Vertex("A"), [
        Vertex("B", 2), 
        Vertex("C", 1), 
        Vertex("D", 3), 
        Vertex("E", 7), 
        Vertex("F", 5)
    ]
)
print(nodes)
nodes.sort()
print(nodes)


A [B w=2, C w=1, D w=3, E w=7, F w=5]
A [C w=1, B w=2, D w=3, F w=5, E w=7]


## saisie des données

In [111]:
# rappel format du noeud : 
# Noeud(nomActuel, [Vertex("nomLié1", poid1),Vertex("nomLié2", poid2),etc...])
data = [
    Node("1",[Vertex("2",2),Vertex("3",1)]),
    Node("2",[Vertex("1",2),Vertex("4",2),Vertex("5",3)]),
    Node("3",[Vertex("1",1),Vertex("2",3),Vertex("4",2)]),
    Node("4",[Vertex("2",2),Vertex("3",5),Vertex("5",2),Vertex("6",4)]),
    Node("5",[Vertex("2",3),Vertex("4",2),Vertex("6",2)]),
    Node("6",[Vertex("4",2),Vertex("5",2)])
]
graphe = Graph(data,"exemple de graphe")

## Kruskal

> init: arrêtes d'ordre ascendant de poids

on peut utiliser `sort()` qui existe a la fois dans python et dans RStudio<br>
> pour i=1...n-1 des sommets<br>
> prendre l'arrête de poids min qui ne fait pas une boucle<br>
> fin

In [112]:
# test
print(graphe)
#graphe.sort_weight_ASC()
#print(graphe)

0| 1 [2, 3]
1| 2 [1, 4, 5]
2| 3 [1, 2, 4]
3| 4 [2, 3, 5, 6]
4| 5 [2, 4, 6]
5| 6 [4, 5]



In [57]:
kruskal <- function(sommets,arretes,poids){
    pooids_ord <- sort(poids,index.return=true)
    poids <- poids_ord$x
    index_poids <- poids_ord$ix 
    return poids
}

SyntaxError: invalid syntax (1399387592.py, line 1)